# LAB: APIs

## Introducción

En este lab vamos a obtener datos usando APIs y con ellos resolver dos problemas:

1. Construir un árbol de regresión que estime la calidad de un vino
2. Analizar las 250 mejores películas de la IMDB

Esto combina lo aprendido sobre los árboles y APIs.
Comenzaremos utilizando [Sheetsu](https://sheetsu.com/your-apis), un servicio gratuito que le permite convertir cualquier hoja de cálculo en una API.
Luego haremos un scraping de datos de [IMDB](http://www.imdb.com/) y utilizaremos los datos obtenidos para analizar las películas de mayor recaudación.

## Ejercicio

### Requisitos

1. Descargar datos desde Sheetsu
- Enviar datos a Sheetsu
- Preparación de los datos
    - Buscar datos faltantes
    - Hacer un análisis exploratorio
- Entrenar un modelo y analizar
- IMDB análisis exploratorio
    1. Obtener las mejores películas de IMDB
    - Obtener los datos de las mejores películas de OMDBAPI
    - Obtener datos de recaudación
    - Preparar los datos:
        - Buscar datos faltantes
        - Usar el tipo correcto de columnas
    - Vectorizar el texto

**Extra:**

- ¿Qué relación hay entre los principales actores y la recaudación de películas?


## Links Útiles

- [IMDB](http://www.imdb.com/)
- [OMDBAPI](http://www.omdbapi.com/)
- [Sheetsu](https://sheetsu.com)

## 1. Obtener datos desde Sheetsu

[Sheetsu](https://sheetsu.com/) es un servicio en línea que le permite acceder a cualquier hoja de cálculo de Google con una API. Es una forma muy poderosa de compartir un conjunto de datos, así como crear un almacenamiento centralizado de datos, que es más sencillo de editar que una base de datos.

Una planilla con datos de vino puede ser encontrada [aquí](https://docs.google.com/spreadsheets/d/1z2awwQ6CsjNA-DFgDAGrcqAfbwnCk4Qiwv92emlqfqk/).

Y puede ser accedida mediante mediante la API de sheetsu [aquí](https://sheetsu.com/apis/v1.0/9afde676cb6b).
Preguntas:

1. Utilizar la librería `requests` para acceder al documento. Inspeccionar la respuesta. ¿Qué tipo de datos son?
> Respuesta: Es un string JSON
2. ¿Cuál es el código de respuesta?
> 200
3. Usar las librerías y funciones apropiadas para leer la respuesta en un Dataframe de pandas
> Respuestas posibles: pd.read_json y json.loads + pd.Dataframe
4. Una vez importados los datos en un dataframe, ¿cuál es el precio del vino en la 6ta línea?
> 6

In [1]:
# Importar los siguientes módulos
import json
import urllib
import pandas as pd
import numpy as np
import requests
import json
import re
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
api_base_url = 'https://sheetsu.com/apis/v1.0/9afde676cb6b'

In [3]:
# 1. Usar la librería `requests` para acceder al documento. Inspeccionar la respuesta. ¿Qué tipo de datos son?
api_response = requests.get(api_base_url)
print("Tipo de datos de la respuesta:", api_response.headers["content-type"])

# 2. ¿Cual es el código de respuesta?
print("Código de respuesta:", api_response.status_code)

Tipo de datos de la respuesta: application/json;charset=UTF-8
Código de respuesta: 200


In [4]:
#3. Usar las librerías y funciones apropiadas para leer la respuesta en un Dataframe de pandas
reponse = json.loads(api_response.text)
wine_df = pd.DataFrame(reponse)
wine_df.head(10)

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,,,,Portugal,4,2013,Vinho Verde
1,W,2015,France,,,17.8,France,3,2013,Peyruchet
2,W,2015,Oregon,,,20,Oregon,3,2013,Abacela
3,W,2015,Spain,chardonay,,7,Spain,2.5,2012,Ochoa
4,R,2015,US,"chiraz, cab",Spice Trader,6,,3,2012,Heartland
5,R,2016,Argentina,malbec,Malbec Argentino,30,Mendoza,4.8,2010,Catena Zapata
6,R,2015,US,,#14,21,Oregon,2.5,2013,Abacela
7,R,2015,France,"merlot, cab",,12,Bordeaux,3.5,2012,David Beaulieu
8,R,2015,France,"merlot, cab",,11.99,Medoc,3.5,2011,Chantemerle
9,R,2015,US,merlot,,13,Washington,4,2011,Hyatt


In [5]:
#3. Otra posibilidad sería leerlo directamente con pandas
wine_df = pd.read_json(api_response.text)
wine_df.head(2)

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,,,,Portugal,4,2013,Vinho Verde
1,W,2015,France,,,17.8,France,3,2013,Peyruchet


In [6]:
#4. Una vez importados los datos en un dataframe. ¿Cuál es el precio del vino en la 6ta linea?
print(wine_df.loc[5,'Price'])

30


## 2. Enviar datos a Sheetsu
Vimos como podemos leer datos, sería genial también poder escribir datos. Para eso haremos un _POST_.

1. Usar el metodo post de `request` para agregar datos a la planilla:
- ¿Cuál fue el status obtenido? ¿Cómo se puede comprobar que se hayan agregado los datos correctamente?
> Respuesta: Enviar y volver a consultar, para verificar que se agregegaron al final
- En este ejercicio, todos están agregando datos a la misma hoja de cálculo, ¿es un problema?, ¿cómo se podría mitigar?
> Sería un problema que cualquiera pueda agregar datos a una hoja de cálculo, en cualquier orden y sin ninguna restricción. Se puede mitigar con permisos específicos.


In [7]:
post_data = {
'Grape' : 'cabernet franc'
, 'Name' : 'Angelica Zapata'
, 'Color' : 'R'
, 'Country' : 'AR'
, 'Region' : 'Mendoza'
, 'Vinyard' : 'Catena Zapata'
, 'Consumed In' : '2017'
, 'Vintage' : '2011'
, 'Price' : '35'
}

In [8]:
ret=requests.post(api_base_url, data=post_data)
print("Código de respuesta:" , ret.status_code)

Código de respuesta: 201


In [9]:
#Verificamos lo que incluimos
vrfy_apiresp = requests.get(api_base_url)
vrfy_df = pd.read_json(vrfy_apiresp.text)
vrfy_df.tail()

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
88,R,2017,AR,cabernet franc,Angelica Zapata,35,Mendoza,,2011,Catena Zapata
89,R,2017,AR,cabernet franc,Angelica Zapata,35,Mendoza,,2011,Catena Zapata
90,R,2017,AR,cabernet franc,Angelica Zapata,35,Mendoza,,2011,Catena Zapata
91,R,2017,AR,cabernet franc,Angelica Zapata,35,Mendoza,,2011,Catena Zapata
92,R,2017,AR,cabernet franc,Angelica Zapata,35,Mendoza,,2011,Catena Zapata


## 3. Preparación de los datos

1. Buscar datos faltantes
    - ¿Hay datos faltantes? ¿Qué se puede hacer?
    - ¿Hay algún dato que pueda eliminar?
    - ¿Los tipos de dato son apropiados?
- Explorar los datos
    - Probar usando describe, min, max, mean, var
- Reducir el dataset a los primeros 29 registros

In [10]:
wine_df.head(1), len(wine_df)

(  Color Consumed In   Country Grape Name Price    Region Score Vintage  \
 0     W        2015  Portugal                   Portugal     4    2013   
 
        Vinyard  
 0  Vinho Verde  , 92)

In [11]:
wine_df = wine_df.replace('', np.nan)

In [12]:
wine_df.head(1)

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,NaN,NaN,NaN,Portugal,4,2013,Vinho Verde


In [13]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 0 to 91
Data columns (total 10 columns):
Color          92 non-null object
Consumed In    87 non-null object
Country        90 non-null object
Grape          81 non-null object
Name           84 non-null object
Price          81 non-null object
Region         89 non-null object
Score          28 non-null object
Vintage        87 non-null object
Vinyard        92 non-null object
dtypes: object(10)
memory usage: 10.4+ KB


In [14]:
wine_df.columns

Index(['Color', 'Consumed In', 'Country', 'Grape', 'Name', 'Price', 'Region',
       'Score', 'Vintage', 'Vinyard'],
      dtype='object')

In [15]:
#wine_df[['Score', 'Price']] = wine_df[['Score', 'Price']].astype(float)
wine_df.loc[wine_df['Price']=='Emiliano', 'Price'] = np.nan
wine_df[['Score', 'Price']] = wine_df[['Score', 'Price']].astype(float)

In [16]:
wine_df.describe()

,Price,Score
count,80.000000,28.000000
mean,37.066125,3.385714
std,36.090961,0.625812
min,6.000000,2.000000
25%,21.750000,3.000000
50%,35.000000,3.500000
75%,35.000000,4.000000
max,235.000000,4.800000


In [17]:
wine_df.head()

,Color,Consumed In,Country,Grape,Name,Price,Region,Score,Vintage,Vinyard
0,W,2015,Portugal,NaN,NaN,NaN,Portugal,4.0,2013,Vinho Verde
1,W,2015,France,NaN,NaN,17.8,France,3.0,2013,Peyruchet
2,W,2015,Oregon,NaN,NaN,20.0,Oregon,3.0,2013,Abacela
3,W,2015,Spain,chardonay,NaN,7.0,Spain,2.5,2012,Ochoa
4,R,2015,US,"chiraz, cab",Spice Trader,6.0,NaN,3.0,2012,Heartland


In [18]:
wine_df=wine_df.head(29)

## 4. IMDB

A veces, una API no proporciona toda la información que nos gustaría obtener y tenemos que ser creativos.
Aquí vamos a usar una combinación de scraping y llamadas API para investigar las calificaciones y ganancias brutas de películas famosas.

## 4.a Obtener las mejores películas de IMDB

La Internet Movie Database contiene datos sobre películas. Desafortunadamente no tiene una API pública.

La página http://www.imdb.com/chart/top contiene la lista de las 250 mejores películas de todos los tiempos. Descargue la página usando `requests` y luego analice el html para obtener una lista de `movie_ids` para estas películas. Puede analizarlo con una expresión regular o utilizando una librería como `BeautifulSoup`.

**Ayuda:** los movie_ids son de este tipo: `tt0068646`

In [19]:
def get_top_250():
    response = requests.get('http://www.imdb.com/chart/top')
    html = response.text
    entries = re.findall("<a href.*?/title/(tt.*?)/", html)
    return list(set(entries))

In [20]:
entries = get_top_250()

In [21]:
print(len(entries))
print(entries[0:10])

250
['tt0435761', 'tt0092067', 'tt1895587', 'tt0091763', 'tt0071853', 'tt0347149', 'tt0167404', 'tt0993846', 'tt0101414', 'tt0052618']


## 4.b Obtener los datos de las mejores películas de OMDBAPI

Aunque IMDB no tiene una API pública, existe una API abierta en http://www.omdbapi.com.   
Atencion: OMDBAPI ha dejado de ser publica, utilizar esta app de heroku la cual expone la misma firma:
https://dsimdbapi.herokuapp.com/?i={imdbid} )

Utilizar esta API para obtener información sobre cada una de las 250 películas que ha extraído en el paso anterior.
1. Leer la documentación de omdbapi.com para saber cómo solicitar datos de película por id (recodar reemplazar www.omdbapi.com por dsimdbapi.herokuapp.com)
- Definir una función que devuelva un objeto python con toda la información para un id dado
- Iterar sobre todos los IDs y almacenar los resultados en una lista de tales objetos
- Crear un Dataframe de pandas de la lista

In [22]:
def get_entry(entry):
    
    try:
        res = requests.get('https://dsimdbapi.herokuapp.com/?i='+entry)
        if res.status_code != 200:
            print(entry, res.status_code)
        else:
            print('.'),
        j = json.loads(res.text)
    except ValueError:
        j = None
    except:
        print("X"),
        j = None
    return j

## 4.c Obtener datos de recaudación

La API de OMDB es útil, pero no proporciona información sobre la recaudación de la película.

1. Escribir una función que recupere la recaudación de la página de la película en imdb.com
- La función debe manejar la excepción de que la página no indique la recaudación
- Recuperar la recaudación y almacenarla en un dataframe separado

In [ ]:
def get_gross(entry):
    try:
        response = requests.get('http://www.imdb.com/title/'+entry+"/business?ref_=tt_dt_bus")
        html = response.text
        gross_list = re.findall('Gross</h5>[^\$]*\$([0-9,.]*)', html)
        gross = int(gross_list[0].replace(',', ''))
        print('.'),
        return gross
    except Exception as ex:
        print()
        print(ex, entry)
        return None

## 4.d Preparación de datos

Teniendo información de las películas e información de recaudación, limpiemos los dos conjuntos de datos.
- Comprobar si hay valores nulos. Tener presente de que pueden parecer strings válidos.
- Convertir las columnas a los formatos apropiados. En particular adaptar:
    - Released
    - Runtime
    - year
    - imdbRating
    - imdbVotes
- Combinar los datos de los dos conjuntos de datos en uno solo

## 4.d Vectorización del texto

Hay varias columnas en los datos que contienen una lista separada por comas de elementos, por ejemplo, la columna `Genre` y la columna `Actors`. 

- Transformar esas columnas en binarias usando el vectorizador de conteo de scikit learn.
- Agregar estas columnas al dataframe combinado.

**Ayuda:** Con el fin de obtener el nombre de los actores a la derecha, tendrá que modificar el `token_pattern` en el `CountVectorizer`.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer()
data = cv.fit_transform(df.Country).todense()
columns = ['Country: '+c for c in cv.get_feature_names()]
countrydf = pd.DataFrame(data, columns=columns)
countrydf.head()

In [ ]:
df1 = pd.concat([df, countrydf], axis = 1)

## 5. Entrenar un modelo y analizar

Nos gustaría usar un árbol de regresión para predecir la calificación de las 250 películas de IMDB. Para ello, primero debemos seleccionar los atributos apropiados.

- Establecer el objetivo como la columna `Score`, eliminar las filas sin `Score`
- Usar pd.get_dummies para crear características dummy para todas las columnas de texto
- Llenar los valores NaN en las columnas numéricas, utilizando un método apropiado
- Entrenar a un Árbol de Regresión sobre `Score`, usando una división de entrenamiento-prueba:
        X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.3, random_state=42)
- Entrenar varios árboles de regresión usando `GridSearchCV` y tuneando los siguientes parámetros `max_leaf_nodes`, `min_samples_leaf` y `min_samples_split`
- Graficar los valores de prueba, los valores predichos y los residuos
- Calcular el error cuadrático medio
- Discutir los hallazgos

In [ ]:
def search_nulls_in_cols(df):
    for i in (df.columns):
        n_nulls = sum(df[i].isnull())
        if n_nulls > 0:
            print(i,n_nulls)           

search_nulls_in_cols(df1)

In [ ]:
df1['Gross'] = df1['Gross'].fillna(df['Gross'].mean())

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline

In [ ]:
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train.head()

In [ ]:
from sklearn.model_selection import GridSearchCV

PARAMETROS = {'max_leaf_nodes':range(2,50), 'min_samples_leaf':range(5,10),
              'min_samples_split': range(2,20)}

#s = np.random.randint(100) # DESCOMENTAR 

#Se hace la búsqueda con Grid Search
model = DecisionTreeRegressor()
gs = GridSearchCV(model, PARAMETROS, scoring='neg_mean_squared_error', verbose=1 , n_jobs=3)
gs.fit(X, y)

# Se muestran los mejores resultados
print(gs.best_estimator_)
print(gs.best_score_)
print(np.sqrt(-gs.best_score_))

Desde la terminal, tenemos que instalar lo siguiente:
> sudo apt-get install python-pydot

o lo siguiente:
> pip install pydotplus

In [ ]:
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus

In [ ]:
# Se grafica el mejor arbol


## Extra:

1. ¿Cuáles son las 10 películas más taquilleras?
- ¿Quiénes son los 10 actores que aparecen en la mayor cantidad de películas?
